### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
from plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "TerraIncognita"
MODEL_NAME = "WinKawaks/vit-tiny-patch16-224"

c:\Users\Fatim_Sproj\anaconda3\envs\bacp\lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-20 04:05:08,518 | INFO | Initialized experiment directories for TerraIncognita
2025-10-20 04:05:08,518 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\TerraIncognita\logs
2025-10-20 04:05:08,518 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\TerraIncognita\checkpoints
2025-10-20 04:05:08,518 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\TerraIncognita\plots


### Setup

In [4]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]


{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [5]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-20 04:05:10,149 | INFO | === LODO: Leaving out domain 'location_38' ===



=== LODO: Leaving out domain 'location_38' ===


Evaluating: 100%|██████████| 77/77 [01:26<00:00,  1.13s/it]
2025-10-20 04:08:10,547 | INFO | [location_38] Epoch 1/5 | Train - Loss: 1.2368, Cls: 1.2291, GRQO: 0.0077, Acc: 0.5571 | Val - Loss: 2.7534, Cls: 2.7525, GRQO: 0.0009, Acc: 0.1734
2025-10-20 04:08:10,597 | INFO | [location_38] New best val acc: 0.1734


[location_38] Epoch 1/5 | Train - Loss: 1.2368, Cls: 1.2291, GRQO: 0.0077, Acc: 0.5571 | Val - Loss: 2.7534, Cls: 2.7525, GRQO: 0.0009, Acc: 0.1734
[location_38] New best val acc: 0.1734


Evaluating: 100%|██████████| 77/77 [00:54<00:00,  1.42it/s]
2025-10-20 04:10:43,238 | INFO | [location_38] Epoch 2/5 | Train - Loss: 0.6373, Cls: 0.6344, GRQO: 0.0029, Acc: 0.7854 | Val - Loss: 3.2079, Cls: 3.2072, GRQO: 0.0006, Acc: 0.1986
2025-10-20 04:10:43,285 | INFO | [location_38] New best val acc: 0.1986


...

[location_100] Epoch 3/5 | Train - Loss: 0.3550, Cls: 0.3526, GRQO: 0.0023, Acc: 0.8872 | Val - Loss: 1.7553, Cls: 1.7548, GRQO: 0.0006, Acc: 0.5144
[location_100] New best val acc: 0.5144


Evaluating: 100%|██████████| 39/39 [00:33<00:00,  1.16it/s]
2025-10-20 05:29:40,613 | INFO | [location_100] Epoch 4/5 | Train - Loss: 0.2435, Cls: 0.2415, GRQO: 0.0019, Acc: 0.9197 | Val - Loss: 2.7837, Cls: 2.7833, GRQO: 0.0004, Acc: 0.3603


[location_100] Epoch 4/5 | Train - Loss: 0.2435, Cls: 0.2415, GRQO: 0.0019, Acc: 0.9197 | Val - Loss: 2.7837, Cls: 2.7833, GRQO: 0.0004, Acc: 0.3603


Evaluating: 100%|██████████| 39/39 [00:30<00:00,  1.28it/s]
2025-10-20 05:33:33,107 | INFO | [location_100] Epoch 5/5 | Train - Loss: 0.1735, Cls: 0.1719, GRQO: 0.0016, Acc: 0.9441 | Val - Loss: 2.4784, Cls: 2.4781, GRQO: 0.0003, Acc: 0.3970
2025-10-20 05:33:33,107 | INFO | [location_100] Best Acc: 0.5144
2025-10-20 05:33:33,107 | INFO | ------------------------------------------------------------
2025-10-20 05:33:33,109 | INFO | LODO finished | Mean Acc: 0.3649 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\TerraIncognita\logs\lodo_summary_20251020_053333.json


[location_100] Epoch 5/5 | Train - Loss: 0.1735, Cls: 0.1719, GRQO: 0.0016, Acc: 0.9441 | Val - Loss: 2.4784, Cls: 2.4781, GRQO: 0.0003, Acc: 0.3970
[location_100] Best Acc: 0.5144
------------------------------------------------------------
LODO finished | Mean Acc: 0.3649
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\TerraIncognita\logs\lodo_summary_20251020_053333.json


### Baseline

In [6]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)


2025-10-20 05:33:33,130 | INFO | Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([10, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-20 05:33:34,849 | INFO | === Baseline LODO: Leaving out domain 'location_38' ===



=== Baseline LODO: Leaving out domain 'location_38' ===


2025-10-20 05:36:36,074 | INFO | [location_38] Epoch 1/5 | Train - Loss: 0.9253, Acc: 0.6695 | Val Acc: 0.1073


...

[location_100] Epoch 3/5 | Train - Loss: 0.2281, Acc: 0.9241 | Val Acc: 0.4409


2025-10-20 06:37:24,353 | INFO | [location_100] Epoch 4/5 | Train - Loss: 0.1548, Acc: 0.9486 | Val Acc: 0.4595


[location_100] Epoch 4/5 | Train - Loss: 0.1548, Acc: 0.9486 | Val Acc: 0.4595


2025-10-20 06:42:53,113 | INFO | [location_100] Epoch 5/5 | Train - Loss: 0.1207, Acc: 0.9593 | Val Acc: 0.4441
2025-10-20 06:42:53,114 | INFO | [location_100] Best Val Acc: 0.4595
2025-10-20 06:42:53,115 | INFO | ------------------------------------------------------------
2025-10-20 06:42:53,116 | INFO | Baseline LODO (WinKawaks/vit-tiny-patch16-224) finished | Mean Acc: 0.3675


[location_100] Epoch 5/5 | Train - Loss: 0.1207, Acc: 0.9593 | Val Acc: 0.4441
[location_100] Best Val Acc: 0.4595
------------------------------------------------------------
Baseline LODO (WinKawaks/vit-tiny-patch16-224) finished | Mean Acc: 0.3675
